# **Модель, распознающая эмоции**
Здесь описано создание и обучение модели, распознающей эмоции по фотографии.

## Эмоции
Модель распознает 7 базовых эмоций по Экману:
- злость (ANGER)
- отвращение (DISGUST)
- страх (FEAR)
- счастье (HAPPINESS)
- грусть (SAD)
- удивление (SURPRISE)
- нейтральное выражение (NEUTRAL)

Инициализируем массив эмоций

In [0]:
emotions = [(0, "ANGER"), (1, "DISGUST"), (2, "FEAR"), (3, "HAPPINESS"), 
            (4, "SAD"), (5, "SURPRISE"), (6, "NEUTRAL")]

## Датасет
Для обучения, будем использовать датасет fer2013. Найти его можно [здесь](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data). Загрузите его и поместите в папку с этим файлом.
Или можно примонтировать Google Drive, если используется Google Colab.

In [1]:
# Монтирование Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Путь до датасета, укажите свой
fer_dataset_path = "/content/drive/My Drive/university/reko/fer2013.csv"

## Подготовка данных
Для обучения подготавливаем данные. Грузим датасет. Изображения в датасете имеют размер 48х48 пикселей

In [0]:
import pandas as pd
import numpy as np

In [0]:
width, height = 48, 48
data = pd.read_csv(fer_dataset_path)

Данные в датасете хранятся в виде векторов целых чисел. В столбце emotion указан идентификатор эмоции (ассоциативный массив определен выше), а pixels - это 2304 (48х48) чисел изображения, пиксели. Пример как это хранится внутри csv файла ниже:

In [17]:
data[:5]

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [0]:
datapoints = data['pixels'].tolist()

all_images = []
for image_xs in datapoints:
  # преобразуем из строки в массив чисел
  xx = [int(image_x) for image_x in image_xs.split(' ')]
  # преобразуем одномерный массив в матрицу
  xx = np.asarray(xx).reshape(width, height)
  # добавляем изображение в список всех изображений
  all_images.append(xx.astype('float32'))

In [0]:
all_images = np.asarray(all_images)
all_images = np.expand_dims(all_images, -1)

## Модель
Для распознавания эмоций на изображении лучше всего использовать сверточную нейронную сеть. Для построения модели будем использовать Keras.

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization